# LightGBM 기반 user별 avg_cart만큼 product 추출 (LB 0.2477)

In [1]:
import numpy as np
import pandas as pd

# 0. Introduction of Data

In [2]:
products = pd.read_csv("data/products.csv")
aisles = pd.read_csv("data/aisles.csv")
departments = pd.read_csv("data/departments.csv")
orders = pd.read_csv("data/orders.csv")
prior = pd.read_csv("data/order_products__prior.csv")
train = pd.read_csv("data/order_products__train.csv")
frame_train = pd.read_csv("data/frame_train.csv")
frame_test = pd.read_csv("data/frame_test.csv")

### orders.csv

In [3]:
print("Size of orders : ", len(orders))
orders.head()

Size of orders :  3421083


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


### order_products_prior.csv

In [4]:
print("Size of prior : ", len(prior))
prior.head()

Size of prior :  32434489


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


### products.csv

In [5]:
print("Size of Products : ", len(products))
products.head()

Size of Products :  49688


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


### frame_train.csv

In [6]:
print("Size of frame of train : ", len(frame_train))
frame_train.head()

Size of frame of train :  8474661


,user_id,order_id,product_id,products
0,1,1187899,17122,0
1,1,1187899,196,1
2,1,1187899,26405,1
3,1,1187899,46149,1
4,1,1187899,14084,0


# 1. Integrate Products Information
* products와 prior에서 얻을 수 있는 제품 관련 정보를 통합
* 기존의 attribute인 product_id, product_name, aisle_id, department_id
* 여기에 prior에서 얻을 수 있는 해당 제품에 판매에 관련된 attribute를 추가
* 총주문량(total_orders), 총재주문량(total_reorders), 재주문율(reorder_rate)을 추가함

In [7]:
productsXprior = pd.DataFrame()
productsXprior['product_id'] = prior.groupby(prior.product_id).size().index
productsXprior['product_orders'] = prior.groupby(prior.product_id).size().values
productsXprior['product_reorders'] = prior['reordered'].groupby(prior.product_id).sum().values
productsXprior['product_reorder_rate'] = (productsXprior.product_reorders / productsXprior.product_orders)
print("Size of productsXprior : ", len(productsXprior))
productsXprior.head()

Size of productsXprior :  49677


,product_id,product_orders,product_reorders,product_reorder_rate
0,1,1852,1136,0.613391
1,2,90,12,0.133333
2,3,277,203,0.732852
3,4,329,147,0.446809
4,5,15,9,0.600000


In [8]:
Products = products.merge(productsXprior, on='product_id', how='outer')
print("Size of Products : ", len(Products))
Products.head()

Size of Products :  49688


,product_id,product_name,aisle_id,department_id,product_orders,product_reorders,product_reorder_rate
0,1,Chocolate Sandwich Cookies,61,19,1852.0,1136.0,0.613391
1,2,All-Seasons Salt,104,13,90.0,12.0,0.133333
2,3,Robust Golden Unsweetened Oolong Tea,94,7,277.0,203.0,0.732852
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,329.0,147.0,0.446809
4,5,Green Chile Anytime Sauce,5,13,15.0,9.0,0.600000


In [9]:
Products[Products.isnull().any(axis=1)]

,product_id,product_name,aisle_id,department_id,product_orders,product_reorders,product_reorder_rate
3629,3630,Protein Granola Apple Crisp,57,14,NaN,NaN,NaN
3717,3718,Wasabi Cheddar Spreadable Cheese,21,16,NaN,NaN,NaN
7044,7045,Unpeeled Apricot Halves in Heavy Syrup,88,13,NaN,NaN,NaN
25382,25383,Chocolate Go Bites,61,19,NaN,NaN,NaN
27498,27499,Non-Dairy Coconut Seven Layer Bar,100,21,NaN,NaN,NaN
36232,36233,Water With Electrolytes,100,21,NaN,NaN,NaN
37702,37703,Ultra Sun Blossom Liquid 90 loads Fabric Enhan...,75,17,NaN,NaN,NaN
43724,43725,Sweetart Jelly Beans,100,21,NaN,NaN,NaN
45970,45971,12 Inch Taper Candle White,101,17,NaN,NaN,NaN
46624,46625,Single Barrel Kentucky Straight Bourbon Whiskey,31,7,NaN,NaN,NaN


In [10]:
Products = Products.replace(np.NaN, 0)
print("Size of Products : ", len(Products))
Products.head()

Size of Products :  49688


,product_id,product_name,aisle_id,department_id,product_orders,product_reorders,product_reorder_rate
0,1,Chocolate Sandwich Cookies,61,19,1852.0,1136.0,0.613391
1,2,All-Seasons Salt,104,13,90.0,12.0,0.133333
2,3,Robust Golden Unsweetened Oolong Tea,94,7,277.0,203.0,0.732852
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,329.0,147.0,0.446809
4,5,Green Chile Anytime Sauce,5,13,15.0,9.0,0.600000


# 2. Integrate_Users_Information
* prior와 orders를 통합(priorXorders)하여 user_id, order_id, product_id를 하나로 모음.
* priorXorders를 기반으로 하여 유저 정보를 담고 있는 Users를 생성.

In [11]:
priorXorders = prior.merge(orders, on='order_id')
print("Size of Prior : ", len(priorXorders))
priorXorders.head()

Size of Prior :  32434489


,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,202279,prior,3,5,9,8.0
1,2,28985,2,1,202279,prior,3,5,9,8.0
2,2,9327,3,0,202279,prior,3,5,9,8.0
3,2,45918,4,1,202279,prior,3,5,9,8.0
4,2,30035,5,0,202279,prior,3,5,9,8.0


In [12]:
user_orders = pd.DataFrame()
user_orders['user_avg_orders_period'] = orders.groupby('user_id')['days_since_prior_order'].mean()
user_orders['user_total_orders'] = orders.groupby('user_id').size() - 1

user_orders['user_id'] = user_orders.index
user_orders.index.name = None

print("Size of user_orders : ", len(user_orders))
user_orders.head()

Size of user_orders :  206209


,user_avg_orders_period,user_total_orders,user_id
1,19.000000,10,1
2,16.285714,14,2
3,12.000000,12,3
4,17.000000,5,4
5,11.500000,4,5


In [13]:
user_prior = pd.DataFrame()
user_prior['user_total_products'] = priorXorders.groupby('user_id').size()
user_prior['ordered_products_set'] = priorXorders.groupby('user_id')['product_id'].apply(set)
user_prior['user_unique_products'] = (user_prior.ordered_products_set.map(len))
user_prior = user_prior[['user_total_products', 'user_unique_products', 'ordered_products_set']]

user_prior['user_id'] = user_prior.index
user_prior.index.name = None

print("Size of user_prior : ", len(user_prior))
user_prior.head()

Size of user_prior :  206209


,user_total_products,user_unique_products,ordered_products_set,user_id
1,59,18,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",1
2,195,102,"{45066, 2573, 18961, 23, 32792, 1559, 22559, 1...",2
3,88,33,"{17668, 44683, 48523, 21903, 14992, 21137, 324...",3
4,18,17,"{21573, 42329, 17769, 35469, 37646, 1200, 1905...",4
5,37,23,"{11777, 40706, 28289, 48775, 20754, 6808, 1398...",5


In [14]:
Users = user_prior.merge(user_orders, on='user_id')
print("Size of users : ", len(Users))
Users.head()

Size of users :  206209


,user_total_products,user_unique_products,ordered_products_set,user_id,user_avg_orders_period,user_total_orders
0,59,18,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",1,19.000000,10
1,195,102,"{45066, 2573, 18961, 23, 32792, 1559, 22559, 1...",2,16.285714,14
2,88,33,"{17668, 44683, 48523, 21903, 14992, 21137, 324...",3,12.000000,12
3,18,17,"{21573, 42329, 17769, 35469, 37646, 1200, 1905...",4,17.000000,5
4,37,23,"{11777, 40706, 28289, 48775, 20754, 6808, 1398...",5,11.500000,4


In [15]:
Users['user_avg_cart'] = (Users.user_total_products / Users.user_total_orders)
print("Size of users : ", len(Users))
Users.head()

Size of users :  206209


,user_total_products,user_unique_products,ordered_products_set,user_id,user_avg_orders_period,user_total_orders,user_avg_cart
0,59,18,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",1,19.000000,10,5.900000
1,195,102,"{45066, 2573, 18961, 23, 32792, 1559, 22559, 1...",2,16.285714,14,13.928571
2,88,33,"{17668, 44683, 48523, 21903, 14992, 21137, 324...",3,12.000000,12,7.333333
3,18,17,"{21573, 42329, 17769, 35469, 37646, 1200, 1905...",4,17.000000,5,3.600000
4,37,23,"{11777, 40706, 28289, 48775, 20754, 6808, 1398...",5,11.500000,4,9.250000


# 3. Train Data Generation
* ### 미리 만들어 둔 frame_train을 기반으로 위의 orders, Products, Users 정보를 통합시켜 train data를 만듬.
* order_prior, order_train은 학습데이터로 포함 X, order_test가 없기 때문에 test데이터에 해당 attribute를 포함시킬 수 없음.

In [16]:
train_df = frame_train.copy()
train_df.set_index(['user_id','order_id','product_id']).head(10)

products
user_id order_id product_id          
1       1187899  17122              0
                 196                1
                 26405              1
                 46149              1
                 14084              0
                 13032              1
                 26088              1
                 39657              1
                 12427              0
                 25133              1

In [17]:
train_df = train_df.merge(Products, how='left', on='product_id')
train_df.head()

,user_id,order_id,product_id,products,product_name,aisle_id,department_id,product_orders,product_reorders,product_reorder_rate
0,1,1187899,17122,0,Honeycrisp Apples,24,4,13880.0,9377.0,0.675576
1,1,1187899,196,1,Soda,77,7,35791.0,27791.0,0.776480
2,1,1187899,26405,1,XL Pick-A-Size Paper Towel Rolls,54,17,1214.0,536.0,0.441516
3,1,1187899,46149,1,Zero Calorie Cola,77,7,8558.0,6953.0,0.812456
4,1,1187899,14084,0,Organic Unsweetened Vanilla Almond Milk,91,16,15935.0,12923.0,0.810982


In [18]:
train_df = train_df.merge(Users, how='left', on='user_id')
train_df.head()

,user_id,order_id,product_id,products,product_name,aisle_id,department_id,product_orders,product_reorders,product_reorder_rate,user_total_products,user_unique_products,ordered_products_set,user_avg_orders_period,user_total_orders,user_avg_cart
0,1,1187899,17122,0,Honeycrisp Apples,24,4,13880.0,9377.0,0.675576,59,18,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",19.0,10,5.9
1,1,1187899,196,1,Soda,77,7,35791.0,27791.0,0.776480,59,18,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",19.0,10,5.9
2,1,1187899,26405,1,XL Pick-A-Size Paper Towel Rolls,54,17,1214.0,536.0,0.441516,59,18,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",19.0,10,5.9
3,1,1187899,46149,1,Zero Calorie Cola,77,7,8558.0,6953.0,0.812456,59,18,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",19.0,10,5.9
4,1,1187899,14084,0,Organic Unsweetened Vanilla Almond Milk,91,16,15935.0,12923.0,0.810982,59,18,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",19.0,10,5.9


In [19]:
train_df = train_df.merge(orders, how='left', on=['user_id','order_id'])
train_df.head()

,user_id,order_id,product_id,products,product_name,aisle_id,department_id,product_orders,product_reorders,product_reorder_rate,...,user_unique_products,ordered_products_set,user_avg_orders_period,user_total_orders,user_avg_cart,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,1187899,17122,0,Honeycrisp Apples,24,4,13880.0,9377.0,0.675576,...,18,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",19.0,10,5.9,train,11,4,8,14.0
1,1,1187899,196,1,Soda,77,7,35791.0,27791.0,0.776480,...,18,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",19.0,10,5.9,train,11,4,8,14.0
2,1,1187899,26405,1,XL Pick-A-Size Paper Towel Rolls,54,17,1214.0,536.0,0.441516,...,18,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",19.0,10,5.9,train,11,4,8,14.0
3,1,1187899,46149,1,Zero Calorie Cola,77,7,8558.0,6953.0,0.812456,...,18,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",19.0,10,5.9,train,11,4,8,14.0
4,1,1187899,14084,0,Organic Unsweetened Vanilla Almond Milk,91,16,15935.0,12923.0,0.810982,...,18,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",19.0,10,5.9,train,11,4,8,14.0


### Extra Feature Engineering

In [20]:
train_df['days_since_ratio'] = train_df.days_since_prior_order / train_df.user_avg_orders_period

### 모든 데이터 통합 완료 & 사용할 attribute 선정

In [21]:
train_df.columns

Index(['user_id', 'order_id', 'product_id', 'products', 'product_name',
       'aisle_id', 'department_id', 'product_orders', 'product_reorders',
       'product_reorder_rate', 'user_total_products', 'user_unique_products',
       'ordered_products_set', 'user_avg_orders_period', 'user_total_orders',
       'user_avg_cart', 'eval_set', 'order_number', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order', 'days_since_ratio'],
      dtype='object')

In [22]:
features = [
    # product information
    'aisle_id', 'department_id', 'product_orders', 'product_reorders', 'product_reorder_rate', 
    # user information
    'user_total_products', 'user_unique_products', 'user_avg_orders_period', 'user_total_orders', 'user_avg_cart', 
    # order information
    'order_hour_of_day', 'days_since_prior_order']

In [23]:
train_X = train_df[features]
train_y = train_df['products']

# 4. Test Data Generation

In [24]:
test_df = frame_test.copy()
test_df = test_df.merge(Products, how='left', on='product_id')
test_df = test_df.merge(Users, how='left', on='user_id')
test_df = test_df.merge(orders, how='left', on=['user_id','order_id'])
test_df['days_since_ratio'] = test_df.days_since_prior_order / test_df.user_avg_orders_period

In [25]:
test_X = test_df[features]

# 5. Learning & Prediction

In [26]:
import lightgbm as lgb

In [27]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss'},
    'num_leaves': 96,
    'max_depth': 10,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.95,
    'bagging_freq': 5
}
ROUNDS = 100

### Learning

In [28]:
train_X_ds = lgb.Dataset(train_X, label=train_y, categorical_feature=['aisle_id','department_id'])

In [29]:
bst = lgb.train(params, train_X_ds, ROUNDS)

### Prediction

In [30]:
result = bst.predict(test_X)

In [31]:
result = pd.concat([frame_test,pd.DataFrame(result, columns=['predict'])], axis=1)

In [32]:
result.head()

,user_id,order_id,product_id,predict
0,3,2774568,17668,0.182468
1,3,2774568,44683,0.134593
2,3,2774568,48523,0.109812
3,3,2774568,21903,0.283384
4,3,2774568,14992,0.155435


# 6. Prediction Probability 기반 최종 products list 생성

In [41]:
sorted_result = result.sort_values(['user_id', 'predict'], ascending=[True,False])

In [42]:
sorted_result.head()

,user_id,order_id,product_id,predict
28,3,2774568,23650,0.309742
10,3,2774568,47766,0.289531
3,3,2774568,21903,0.283384
5,3,2774568,21137,0.276665
14,3,2774568,16797,0.252765


In [43]:
sorted_result.reset_index(inplace=True)
sorted_result.drop(['index'], axis=1, inplace=True)
sorted_result.head()

,user_id,order_id,product_id,predict
0,3,2774568,23650,0.309742
1,3,2774568,47766,0.289531
2,3,2774568,21903,0.283384
3,3,2774568,21137,0.276665
4,3,2774568,16797,0.252765


In [59]:
sorted_result

,user_id,order_id,product_id,predict
0,3,2774568,23650,0.309742
1,3,2774568,47766,0.289531
2,3,2774568,21903,0.283384
3,3,2774568,21137,0.276665
4,3,2774568,16797,0.252765
5,3,2774568,22035,0.250929
6,3,2774568,28373,0.238469
7,3,2774568,42265,0.222555
8,3,2774568,32402,0.208004
9,3,2774568,24010,0.206628


In [67]:
ordered_products = sorted_result.groupby(['user_id', 'order_id'])['product_id'].apply(list)
ordered_products

user_id  order_id
3        2774568     [23650, 47766, 21903, 21137, 16797, 22035, 283...
4        329954      [7350, 19057, 37646, 17769, 36606, 25146, 1186...
6        1528013     [21903, 27521, 8424, 48679, 45007, 38293, 2032...
11       1376945     [13176, 14947, 12384, 44632, 33572, 18465, 479...
12       1356845     [13176, 11520, 22959, 30489, 47144, 37646, 229...
15       2161313     [196, 37710, 48142, 12427, 11266, 10441, 43721...
16       1416320     [24852, 21903, 21137, 30391, 28985, 21616, 120...
19       1735923     [196, 13249, 15131, 28221, 39107, 12108, 43772...
20       1980631       [6184, 22362, 46061, 13575, 13914, 41400, 9387]
22       139655      [13176, 27845, 35221, 32655, 21903, 5212, 5450...
25       1411408     [27521, 10096, 26452, 26128, 22451, 37119, 215...
26       2940603     [14947, 44632, 21709, 19894, 10339, 36316, 752...
28       1192143     [24852, 38689, 5479, 21137, 47209, 21903, 4776...
31       280888      [9292, 196, 1940, 32566, 35914, 45603,

In [92]:
d = dict()
for (uid, oid), row in ordered_products.iteritems():
    avg_cart = Users[Users.user_id == uid]['user_avg_cart'].values[0]
    
    row = row[:int(round(avg_cart))]
    
    for prod in row:    
        try:
            d[oid] += ' ' + str(prod)
        except:
            d[oid] = str(prod)

In [93]:
output = pd.DataFrame.from_dict(d, orient='index')
output.reset_index(inplace=True)
output.columns = ['order_id', 'products']
output.head()

,order_id,products
0,1654153,24852 13176 21137 47766 13806 27511 30233 1905...
1,1572866,19660 47209 47626 44815 27086 42445 37646 2752...
2,1703943,37710
3,1703945,11520 47766 47209 27521 30391
4,3276811,44799 19348 27730 5450 47144 16797 5884 28204


In [94]:
output.to_csv('submission/LightGBM-top_n.csv', index=False)